In [1]:
!curl -L https://github.com/lucidrains/enformer-pytorch/raw/main/data/test-sample.pt -o data/test-sample.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 20.1M  100 20.1M    0     0   752k      0  0:00:27  0:00:27 --:--:--  766k0   984k      0  0:00:20  0:00:10  0:00:10 1091k


In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
from torch.cuda.amp import autocast, GradScaler
from enformer_pytorch import Enformer, GenomeIntervalDataset

# import kipoiseq
# import seaborn as sns
# import matplotlib.pyplot as plt

/Users/williamconnell/miniconda3/envs/eft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Running on MPS:", device)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU:", device)
else:
    device = torch.device("cpu")
    print("Running on CPU:", device)

Running on MPS: mps


In [4]:
TEST_MODEL = True

if TEST_MODEL:
  enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough').to(device)
  data = torch.load('data/test-sample.pt', map_location=device)
  seq, target = data['sequence'].to(device), data['target'].to(device)

  with torch.no_grad():
      corr_coef = enformer(
          seq,
          target = target,
          return_corr_coef = True,
          head = 'human'
      )

  print(corr_coef)
  assert corr_coef > 0.1

/Users/williamconnell/miniconda3/envs/eft/lib/python3.11/site-packages/enformer_pytorch/modeling_enformer.py:82: UserWarning: The operator 'aten::lgamma.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1695391829313/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  log_normalization = (torch.lgamma(concentration) - concentration * torch.log(rate))


tensor(0.5963, device='mps:0')
